# Chapter 29: Backup and Restore Fundamentals

Data durability is the bedrock of production database operations. This chapter provides comprehensive coverage of PostgreSQL backup strategies—from logical dumps suitable for migrations and selective restoration to physical backups for point-in-time recovery. We emphasize industry standards for backup verification, automation, and disaster recovery planning that satisfies enterprise compliance requirements (RTO/RPO objectives).

---

## 29.1 Logical vs. Physical Backup Strategies

PostgreSQL offers two distinct backup paradigms, each serving different operational needs.

### 29.1.1 Logical Backups (SQL Dumps)

**Concept**: Logical backups export database objects (schemas, tables, data) as SQL statements or archive formats. They are portable across PostgreSQL versions and architectures.

**Use Cases**:
- Database migrations (major version upgrades, cloud transitions)
- Selective table restoration
- Schema-only backups for version control
- Small to medium databases (< 1TB)

**Tools**: `pg_dump` (single database), `pg_dumpall` (cluster-wide)

```bash
# Basic logical backup (custom format - compressed, flexible)
pg_dump -h prod-db.example.com -U backup_user -d production \
    -Fc -f production_$(date +%Y%m%d_%H%M%S).dump

# Flags explained:
# -Fc : Custom format (compressed, supports selective restore)
# -Fd : Directory format (parallelizable, one file per table)
# -Fp : Plain text SQL (human readable, larger size)
# -Ft : Tar format (legacy, limited functionality)
```

### 29.1.2 Physical Backups (Filesystem Copy)

**Concept**: Physical backups copy the actual data files (heap files, WAL segments) at the filesystem level. They preserve the exact binary state of the cluster.

**Use Cases**:
- Large databases (> 1TB) where logical backup is too slow
- Point-in-time recovery (PITR) requirements
- Disaster recovery with minimal RTO
- Replica seeding

**Tools**: `pg_basebackup`, filesystem snapshots (LVM, ZFS, EBS, Azure Disk)

```bash
# Basic physical backup (base backup for replication/PITR)
pg_basebackup -h prod-db.example.com -U replicator \
    -D /backups/base/$(date +%Y%m%d) \
    -Fp -Xs -P -v

# Flags explained:
# -D : Destination directory
# -Fp : Plain format (copy files as-is)
# -Ft : Tar format (single tar file per tablespace)
# -Xs : Stream WAL during backup (essential for consistency)
# -P : Progress reporting
# -v : Verbose
```

### 29.1.3 Strategic Comparison

| Characteristic | Logical (pg_dump) | Physical (pg_basebackup) |
|----------------|-------------------|--------------------------|
| **Portability** | Cross-version, cross-architecture | Same version, same architecture |
| **Size** | Compressed, smaller (no indexes bloat) | Exact copy including bloat |
| **Granularity** | Table-level, schema-level | Entire cluster only |
| **Recovery Speed** | Slow (replays SQL) | Fast (filesystem copy) |
| **PITR Support** | No | Yes (with WAL archiving) |
| **Parallelism** | Parallel dump/restore | Parallel compression |
| **Impact** | Heavy query load (long transactions) | Minimal (brief checkpoint) |

**Industry Standard**: Hybrid approach—logical backups for migrations and table-level recovery, physical backups for disaster recovery and PITR.

---

## 29.2 Logical Backup Mastery with pg_dump

### 29.2.1 Backup Formats and Selection

PostgreSQL supports four output formats, each with specific operational characteristics.

**Custom Format (-Fc) - Recommended Default**:

```bash
# Custom format backup (compressed, selective restore capable)
pg_dump -h localhost -U postgres -d production \
    -Fc -Z9 -j 4 \
    -f /backups/prod_$(date +%Y%m%d_%H%M%S).dump

# -Z9 : Compression level 9 (highest)
# -j 4 : Parallel dump jobs (requires directory format for tables, or pg_dump 12+)
```

**Directory Format (-Fd) - Parallel Operations**:

```bash
# Directory format (best for large databases)
pg_dump -h localhost -U postgres -d production \
    -Fd -j 8 -Z 5 \
    -f /backups/prod_dir_$(date +%Y%m%d)

# Structure:
# /backups/prod_dir_20241002/
#   ├── toc.dat (table of contents)
#   ├── 1234.dat.gz (table data)
#   ├── 1235.dat.gz (table data)
#   └── ...

# Advantages:
# 1. Parallel dump (-j 8 uses 8 connections)
# 2. Parallel restore (pg_restore -j 8)
# 3. Individual file compression
```

**Plain SQL Format (-Fp) - Version Control**:

```bash
# Schema-only dump for version control (no data)
pg_dump -h localhost -U postgres -d production \
    -Fp --schema-only \
    --no-owner --no-privileges \
    -f schema_$(date +%Y%m%d).sql

# --no-owner : Remove "ALTER OWNER" commands (useful for restoring to different owner)
# --no-privileges : Skip GRANT/REVOKE (useful for security sanitization)
```

### 29.2.2 Selective Backups (Object Filtering)

For large databases, backing up specific schemas or tables reduces time and storage.

```bash
# Backup specific schemas only
pg_dump -h localhost -U postgres -d production \
    -Fc \
    --schema=public \
    --schema=reporting \
    -f schemas_core.backup

# Exclude specific schemas (data warehousing pattern)
pg_dump -h localhost -U postgres -d production \
    -Fc \
    --exclude-schema=analytics_temp \
    --exclude-schema=cache_tables \
    -f prod_without_temp.backup

# Table-level backup (critical tables)
pg_dump -h localhost -U postgres -d production \
    -Fc \
    --table=users \
    --table=orders \
    --table=payments \
    -f critical_tables.backup

# Pattern matching for tables
pg_dump -h localhost -U postgres -d production \
    -Fc \
    --table='public.*_audit' \
    --table='public.log_*' \
    -f audit_tables.backup

# Exclude large non-critical tables
pg_dump -h localhost -U postgres -d production \
    -Fc \
    --exclude-table=session_logs \
    --exclude-table=page_view_events \
    -f prod_no_logs.backup
```

### 29.2.3 Advanced pg_dump Options

**Consistent Snapshots (Critical for Transactional Integrity)**:

```bash
# Ensure snapshot consistency across parallel jobs
pg_dump -h localhost -U postgres -d production \
    -Fd -j 4 \
    --snapshot=snapshot_name \  # Use with caution, requires transaction management
    -f /backups/consistent_backup

# Note: Without --snapshot, each parallel job gets its own transaction snapshot
# For true consistency, use single-threaded or ensure no DDL during backup
```

**Data-Only vs Schema-Only**:

```bash
# Schema only (for dev environment setup)
pg_dump --schema-only --no-owner -f schema.sql

# Data only (after schema migration)
pg_dump --data-only --disable-triggers -f data.sql
# --disable-triggers : Adds commands to disable/enable triggers during load
# Useful for avoiding FK constraint violations during restore
```

**Section-Specific Restoration Support**:

```bash
# Backup with pre-data, data, and post-data sections clearly separated
pg_dump -Fc -f backup.dump

# Later restore in stages:
# 1. Schema (tables, constraints without FK checks)
# 2. Data (bulk load)
# 3. Post-data (indexes, FK constraints, triggers)
```

---

## 29.3 Restoration with pg_restore

### 29.3.1 Restoration Strategies

**List and Selective Restore**:

```bash
# Inspect backup contents without restoring
pg_restore -l backup.dump > backup_contents.txt

# Contents example:
# 2; 145344 TABLE production users postgres
# 3; 145345 TABLE production orders postgres
# 4; 145346 SEQUENCE production orders_order_id_seq postgres

# Restore specific tables only
pg_restore -d target_db \
    --table=users \
    --table=orders \
    backup.dump

# Restore specific schemas
pg_restore -d target_db \
    --schema=public \
    backup.dump
```

**Parallel Restoration (Fastest for Large Databases)**:

```bash
# Parallel restore (requires directory format backup)
pg_restore -d target_db \
    -j 8 \              # 8 parallel jobs (adjust to CPU count)
    --no-owner \        # Don't preserve original ownership
    --no-privileges \   # Don't preserve privileges (security sanitization)
    --clean \           # Drop objects before recreating
    --if-exists \       # Add IF EXISTS to DROP commands
    /backups/prod_dir_20241002

# Performance tuning:
# -j should match available CPU cores on target
# Ensure target has sufficient IOPS (SSD recommended)
```

**Targeted Restoration to Different Database**:

```bash
# Restore to different database name
createdb -U postgres new_production
pg_restore -d new_production \
    --no-owner \
    --role=new_owner \  # Change ownership to new role
    backup.dump

# Restore to different schema (remapping)
pg_restore -d target_db \
    --schema=old_schema \
    backup.dump
# Then: ALTER SCHEMA old_schema RENAME TO new_schema;
```

### 29.3.2 Handling Restoration Errors

**Single Transaction vs. Per-Statement**:

```bash
# Default: Each command in separate transaction (continues on error)
pg_restore -d target_db backup.dump

# Strict mode: All or nothing (stop on first error)
pg_restore -d target_db --single-transaction backup.dump
# Note: Only works if backup is small enough for single transaction

# Continue on errors but log them
pg_restore -d target_db \
    --exit-on-error \  # Stop on error (default is to continue)
    -e \               # Exit on error (alternative flag)
    backup.dump 2> restore_errors.log
```

**Disable Triggers During Data Load**:

```bash
# Speed up data-only restore by disabling triggers
pg_restore -d target_db \
    --data-only \
    --disable-triggers \  # Disables triggers during load, reenables after
    backup.dump

# Caution: Bypasses foreign key constraints and business logic in triggers
# Only use when you trust data integrity from source
```

---

## 29.4 Physical Backup with pg_basebackup

### 29.4.1 Preparing for Physical Backups

**Prerequisites**:

1. **Replication User** (for pg_basebackup):
```sql
-- Create dedicated backup user
CREATE USER backup_user WITH REPLICATION ENCRYPTED PASSWORD 'secure_random_string';
-- OR use existing replicator role

-- pg_hba.conf entry (restrict to backup server IP)
hostssl replication backup_user 10.0.0.10/32 scram-sha-256
```

2. **WAL Archiving Configuration** (postgresql.conf):
```ini
# Enable WAL archiving for PITR capability
wal_level = replica              # or 'logical' if needed
archive_mode = on
archive_command = 'cp %p /backups/wal/%f'  # Simple file copy (use rsync/scp for remote)
# Or use cloud storage:
# archive_command = 'aws s3 cp %p s3://bucket/wal/%f'

# Alternative: archive_library for PostgreSQL 15+ (pgbackrest, barman, wal-g)
```

### 29.4.2 Executing Physical Backups

**Standard Base Backup**:

```bash
# Create base backup directory
mkdir -p /backups/base/$(date +%Y%m%d_%H%M%S)

# Execute backup
pg_basebackup \
    -h prod-db.example.com \
    -U backup_user \
    -D /backups/base/$(date +%Y%m%d_%H%M%S) \
    -Fp \           # Plain format (files as-is)
    -Xs \           # Stream WAL (essential for consistency)
    -P \            # Show progress
    -v \            # Verbose
    -W \            # Force password prompt (or use .pgpass)
    --checkpoint=fast \  # Request immediate checkpoint (don't wait for spread)
    --wal-method=stream   # Stream WAL during backup (fetch for older versions)

# Result:
# /backups/base/20241002_143022/
# ├── base/           # Data files
# ├── global/         # Global objects
# ├── pg_wal/         # WAL files streamed during backup
# ├── backup_label    # Backup metadata (START WAL location, time)
# └── tablespace_map  # Tablespace mappings
```

**Compressed Physical Backup**:

```bash
# Tar format with compression (single file, easier to move)
pg_basebackup \
    -h prod-db.example.com \
    -U backup_user \
    -Ft \           # Tar format
    -z \            # Compress with gzip (-Z for level)
    -D - \          # Output to stdout
    -Xs > /backups/base/base_$(date +%Y%m%d).tar.gz

# Extract when needed:
# tar -xzf base_20241002.tar.gz -C /var/lib/postgresql/data
```

**Tablespace Handling**:

```bash
# Backup with separate tablespaces
pg_basebackup \
    -D /backups/base/main \
    --tablespace-mapping=/ssd/data=/backups/base/ssd_data \
    --tablespace-mapping=/hdd/archive=/backups/base/hdd_archive \
    -Fp -Xs

# Maps:
# /ssd/data -> /backups/base/ssd_data
# /hdd/archive -> /backups/base/hdd_archive
```

### 29.4.3 Incremental Physical Backups (PostgreSQL 17+)

PostgreSQL 17 introduces native incremental backup support using WAL summarization.

```ini
# postgresql.conf (PostgreSQL 17+)
summarize_wal = on  # Enable WAL summarization for incremental backups
```

```bash
# Full backup (every Sunday)
pg_basebackup -D /backups/base/full_$(date +%Y%m%d) -Fp -Xs

# Incremental backup (daily)
pg_basebackup \
    --incremental=/backups/base/full_20241001/backup_manifest \
    -D /backups/inc/inc_$(date +%Y%m%d) \
    -Fp

# Restore combines full + incrementals automatically via manifest
```

---

## 29.5 Backup Verification and Testing

### 29.5.1 The Restore Drill (Industry Standard)

**Untested backups are Schrodinger's backups—they simultaneously exist and don't exist until observed.**

**Automated Verification Process**:

```bash
#!/bin/bash
# restore_test.sh - Automated backup verification

BACKUP_FILE="/backups/prod_$(date +%Y%m%d).dump"
TEST_DB="restore_test_$(date +%s)"
PGHOST="localhost"
PGUSER="postgres"

# 1. Create ephemeral test database
createdb $TEST_DB

# 2. Restore backup
pg_restore -d $TEST_DB --no-owner --no-privileges $BACKUP_FILE
RESTORE_STATUS=$?

if [ $RESTORE_STATUS -ne 0 ]; then
    echo "ERROR: Restore failed with status $RESTORE_STATUS"
    dropdb $TEST_DB
    exit 1
fi

# 3. Verify row counts match expected ranges
psql -d $TEST_DB -c "
    SELECT 
        'users' as table_name, 
        CASE WHEN COUNT(*) BETWEEN 100000 AND 200000 THEN 'PASS' ELSE 'FAIL' END as status,
        COUNT(*) as actual_count
    FROM users
    UNION ALL
    SELECT 
        'orders', 
        CASE WHEN COUNT(*) > 500000 THEN 'PASS' ELSE 'FAIL' END,
        COUNT(*)
    FROM orders;
" > verification_report.txt

# 4. Check for constraint violations
psql -d $TEST_DB -c "
    SELECT 
        conrelid::regclass as table_name,
        conname as constraint_name
    FROM pg_constraint
    WHERE convalidated = false;
" > constraint_check.txt

# 5. Verify critical application query works
psql -d $TEST_DB -c "
    EXPLAIN (ANALYZE, BUFFERS) 
    SELECT * FROM users WHERE email = 'test@example.com';
" > performance_check.txt

# 6. Cleanup
dropdb $TEST_DB

# 7. Report
echo "Backup verification completed. See verification_report.txt"
```

**Physical Backup Verification**:

```bash
# Verify base backup integrity
pg_verifybackup /backups/base/20241002_143022

# Checks:
# - Files exist and match manifest
# - Checksums match (if checksums enabled)
# - WAL files complete
# - backup_label valid

# Expected output:
# backup successfully verified
```

### 29.5.2 Checksums and Integrity

Enable data checksums during initdb (cannot be enabled on existing cluster without dump/restore):

```bash
# Initialize cluster with checksums (essential for corruption detection)
initdb -D /var/lib/postgresql/data --data-checksums

# Verify checksums in backup
pg_checksums --check -D /backups/base/20241002_143022
```

---

## 29.6 Handling Large Databases

### 29.6.1 Parallel Dump Strategy

For multi-terabyte databases, single-threaded dumps are impractical.

```bash
# Step 1: Schema backup (fast, single-threaded)
pg_dump -h prod -U postgres -d bigdb \
    --schema-only \
    -f bigdb_schema.sql

# Step 2: Parallel data dump (directory format)
pg_dump -h prod -U postgres -d bigdb \
    -Fd -j 16 \      # 16 parallel jobs
    --data-only \
    -f /mnt/fast_storage/bigdb_data

# Optimization flags:
--no-sync \          # Don't fsync each table (faster, risk of OS crash losing backup)
--lock-wait-timeout=5000 \  # Fail fast if can't acquire lock (ms)
```

### 29.6.2 Object-Specific Strategies

**Partitioned Tables**:

```bash
# Backup recent partitions only (rolling window)
pg_dump -d bigdb \
    --table='orders_2024*' \
    --table='events_2024*' \
    -Fc -f recent_partitions.dump

# Archive old partitions separately (rarely accessed)
pg_dump -d bigdb \
    --table='orders_2022*' \
    --table='orders_2023*' \
    -Fc -f old_partitions.dump
# Store old_partitions in cold storage (Glacier, etc.)
```

**BLOB/TEXT Exclusions**:

```bash
# Exclude large text columns or TOAST tables if data available elsewhere
pg_dump -d bigdb \
    --exclude-table-data=large_text_storage \
    --exclude-table-data=image_binaries \
    -Fc -f main_data.dump

# Backup large objects separately with compression
pg_dump -d bigdb \
    --blobs \
    --format=custom \
    -Z9 \
    -f blobs_only.dump
```

### 29.6.3 Physical Backup for Very Large Databases

When logical backup exceeds acceptable windows (> 8 hours):

```bash
# Continuous archiving with pgBackRest (industry standard tool)
# pgBackRest configuration (/etc/pgbackrest/pgbackrest.conf)
[global]
repo1-path=/backups/pgbackrest
repo1-retention-full=2
repo1-retention-diff=4
process-max=8

[production]
pg1-path=/var/lib/postgresql/15/main
pg1-port=5432

# Commands:
# Full backup (weekly)
pgbackrest --stanza=production backup --type=full

# Incremental (daily)
pgbackrest --stanza=production backup --type=incr

# Differential (mid-week)
pgbackrest --stanza=production backup --type=diff

# Restore to specific point in time
pgbackrest --stanza=production restore \
    --target="2024-10-02 14:30:00" \
    --target-action=promote
```

---

## 29.7 Automation and Scheduling

### 29.7.1 Logical Backup Automation

```bash
#!/bin/bash
# /usr/local/bin/pg_backup_rotated.sh

# Configuration
BACKUP_DIR="/backups/daily"
RETENTION_DAYS=7
DB_NAME="production"
DB_HOST="localhost"
DATE=$(date +%Y%m%d_%H%M%S)
BACKUP_FILE="$BACKUP_DIR/${DB_NAME}_${DATE}.dump"

# Ensure directory exists
mkdir -p $BACKUP_DIR

# Execute backup with error handling
if ! pg_dump -h $DB_HOST -U backup_user -d $DB_NAME \
    -Fc -Z9 \
    -f $BACKUP_FILE 2>> $BACKUP_DIR/backup.log; then
    echo "Backup failed at $(date)" | mail -s "Backup Alert" dba@company.com
    exit 1
fi

# Verify file size (basic sanity check)
FILE_SIZE=$(stat -c%s "$BACKUP_FILE")
if [ $FILE_SIZE -lt 1024 ]; then
    echo "Backup file suspiciously small: $FILE_SIZE bytes" | mail -s "Backup Alert" dba@company.com
    rm $BACKUP_FILE
    exit 1
fi

# Encrypt backup (optional but recommended for compliance)
gpg --encrypt --recipient backup@company.com --trust-model always \
    --output ${BACKUP_FILE}.gpg $BACKUP_FILE
rm $BACKUP_FILE  # Remove unencrypted

# Cleanup old backups
find $BACKUP_DIR -name "*.gpg" -mtime +$RETENTION_DAYS -delete

# Sync to remote (S3 example)
aws s3 sync $BACKUP_DIR s3://company-backups/postgres/daily/ \
    --exclude "*" --include "*.gpg"
```

**Cron Scheduling**:

```cron
# Daily logical backup at 2 AM (low traffic period)
0 2 * * * postgres /usr/local/bin/pg_backup_rotated.sh

# Weekly full physical backup (Sunday 3 AM)
0 3 * * 0 postgres pg_basebackup -D /backups/weekly/$(date +\%Y\%m\%d) -Fp -Xs -P
```

### 29.7.2 WAL Archiving Automation

```bash
# archive_command for S3 (using wal-e or aws cli)
archive_command = 'test ! -f /backups/wal/%f && cp %p /backups/wal/%f; aws s3 cp /backups/wal/%f s3://bucket/wal/ && rm /backups/wal/%f'

# Or use wal-g (modern Go-based tool)
archive_command = 'wal-g wal-push %p'
restore_command = 'wal-g wal-fetch %f %p'
```

---

## 29.8 Security and Compliance Considerations

### 29.8.1 Backup Encryption

**At-Rest Encryption**:

```bash
# Encrypt during creation (gpg)
pg_dump -d production -Fc | \
    gpg --encrypt --recipient backup@company.com \
    > production_$(date +%Y%m%d).dump.gpg

# Decrypt for restore
gpg --decrypt production_20241002.dump.gpg | \
    pg_restore -d target_db
```

**Storage Encryption**:
- Use encrypted volumes (LUKS, BitLocker, AWS EBS encryption, Azure Disk Encryption)
- Ensure backups on S3 use SSE-S3 or SSE-KMS
- Maintain encryption key rotation policies

### 29.8.2 Access Control

```sql
-- Backup user with minimal privileges (can read all data but not modify)
CREATE USER backup_user WITH REPLICATION LOGIN PASSWORD '...';

-- Grant SELECT on all tables (for logical backup)
GRANT SELECT ON ALL TABLES IN SCHEMA public TO backup_user;
GRANT SELECT ON ALL TABLES IN SCHEMA analytics TO backup_user;

-- For pg_dumpall (globals only), superuser is unfortunately required
-- Alternative: dump globals separately with controlled script
```

### 29.8.3 Compliance Checklist (SOC 2, HIPAA, PCI)

- [ ] **Encryption**: Backups encrypted at rest (AES-256) and in transit (TLS 1.2+)
- [ ] **Retention**: Defined retention periods enforced (automated deletion after period)
- [ ] **Immutability**: Write-once storage (S3 Object Lock, WORM tape) for ransomware protection
- [ ] **Testing**: Quarterly restore drills documented with RTO/RTO measurements
- [ ] **Access Logging**: Who accessed backups, when, and from where
- [ ] **Offsite**: Geographic separation of backups (different availability zone/region)
- [ ] **Sanitization**: Non-production restores use masked/anonymized data where required

---

## Chapter Summary

In this chapter, you learned:

1. **Backup Types**: Logical backups (`pg_dump`) provide portability and granularity for migrations and table-level recovery; physical backups (`pg_basebackup`) provide fast disaster recovery and point-in-time recovery capability. Use logical for <1TB databases and migrations, physical for large production databases.

2. **pg_dump Best Practices**: Use custom format (`-Fc`) for flexibility, directory format (`-Fd`) with parallel jobs (`-j`) for large databases, and selective filtering (`--table`, `--exclude-table`) to manage size. Always use `--no-owner` and `--no-privileges` when restoring to different environments.

3. **pg_restore Techniques**: Leverage parallel restoration (`-j`) for speed, use `--single-transaction` for atomicity on small databases, and employ `--disable-triggers` carefully during data-only restores. Inspect backups with `-l` before selective restoration.

4. **Physical Backup Architecture**: Configure `wal_level = replica` and `archive_mode = on` for PITR. Use `pg_basebackup` with `-Xs` to stream WAL during backup ensuring consistency. Store WAL archives separately from base backups.

5. **Verification**: Implement automated restore drills that create ephemeral databases, verify row counts, check constraint validity, and test application connectivity. Use `pg_verifybackup` for physical backups. Untested backups are invalid.

6. **Large Database Strategy**: For multi-terabyte systems, use parallel directory format dumps, partition-specific backups (archiving old partitions separately), or physical backup tools like pgBackRest. Consider excluding large binary columns if separately stored.

7. **Security**: Encrypt backups at rest using GPG or storage-level encryption (LUKS/SSE-KMS). Restrict backup user privileges to `REPLICATION` and `SELECT`. Maintain offsite, immutable backups (WORM) for ransomware protection and compliance.

**Next**: In Chapter 30, we will explore Point-in-Time Recovery (PITR) and Write-Ahead Log (WAL) architecture—covering WAL archiving strategies, recovery target specifications, timeline management, and achieving specific RPO (Recovery Point Objective) targets through continuous archiving.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../7. security_and_access_control_industry_baselines/28. application_security_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='30. point_in_time_recovery_pitr_and_wal.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
